<a href="https://colab.research.google.com/github/fnko19/Deteksi-Lubang-Jalan/blob/main/DetectionPlotholeCanny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Keseluruhan Code


In [ ]:
import cv2
import numpy as np

def manual_canny_improved(img, low_thresh=30, high_thresh=100):
    Kx = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], dtype=np.float32)
    Ky = np.array([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]], dtype=np.float32)

    Ix = cv2.filter2D(img, cv2.CV_64F, Kx)
    Iy = cv2.filter2D(img, cv2.CV_64F, Ky)

    G = np.hypot(Ix, Iy)
    G = G.astype(np.uint8)
    theta = np.arctan2(Iy, Ix)

    M, N = img.shape
    Z = np.zeros((M, N), dtype=np.uint8)
    angle = theta * 180. / np.pi
    angle[angle < 0] += 180

    for i in range(1, M - 1):
        for j in range(1, N - 1):
            q = 255
            r = 255

            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = G[i, j + 1]
                r = G[i, j - 1]
            elif (22.5 <= angle[i, j] < 67.5):
                q = G[i + 1, j - 1]
                r = G[i - 1, j + 1]
            elif (67.5 <= angle[i, j] < 112.5):
                q = G[i + 1, j]
                r = G[i - 1, j]
            elif (112.5 <= angle[i, j] < 157.5):
                q = G[i - 1, j - 1]
                r = G[i + 1, j + 1]

            if (G[i, j] >= q) and (G[i, j] >= r):
                Z[i, j] = G[i, j]
            else:
                Z[i, j] = 0

    # Double Threshold
    strong, weak = 255, 75
    res = np.zeros_like(Z, dtype=np.uint8)
    strong_i, strong_j = np.where(Z >= high_thresh)
    weak_i, weak_j = np.where((Z >= low_thresh) & (Z < high_thresh))

    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak

    # Hysteresis
    for i in range(1, M - 1):
        for j in range(1, N - 1):
            if res[i, j] == weak:
                if np.any(res[i - 1:i + 2, j - 1:j + 2] == strong):
                    res[i, j] = strong
                else:
                    res[i, j] = 0

    return res

# === Proses Seluruh Video ===
video_path = '/content/drive/MyDrive/p.mp4'
output_path = '/content/pothole_detected_canny_manual.mp4'

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #equalized = cv2.equalizeHist(gray) <---- aktifkan ini KHUSUS kalau mau mencoba dengan Histogram Equalization (matikan kalau tidak)
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)) <---- aktifkan ini KHUSUS kalau mau mencoba dengan CLAHE (matikan kalau tidak)
    #equalized = clahe.apply(gray) <---- aktifkan ini kalau KHUSUS mau mencoba dengan CLAHE (matikan kalau tidak)
    blurred = cv2.GaussianBlur(gray, (5, 5), sigmaX=1.4)

    edges = manual_canny_improved(blurred, 30, 100)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)
        rect_area = w * h
        extent = area / rect_area if rect_area > 0 else 0
        aspect_ratio = w / float(h)

        if area > 150 and extent > 0.4 and 1.5 <= aspect_ratio <= 10.0:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, "lubang", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)

    out.write(frame)
    frame_count += 1
    if frame_count % 10 == 0:
        print(f"Memproses frame ke-{frame_count}...")

cap.release()
out.release()
print("Selesai! Video disimpan di:", output_path)


🧩 Memproses frame ke-10...
🧩 Memproses frame ke-20...
🧩 Memproses frame ke-30...
🧩 Memproses frame ke-40...
🧩 Memproses frame ke-50...
🧩 Memproses frame ke-60...
🧩 Memproses frame ke-70...
🧩 Memproses frame ke-80...
🧩 Memproses frame ke-90...
🧩 Memproses frame ke-100...
🧩 Memproses frame ke-110...
🧩 Memproses frame ke-120...
🧩 Memproses frame ke-130...
🧩 Memproses frame ke-140...
🧩 Memproses frame ke-150...
🧩 Memproses frame ke-160...
🧩 Memproses frame ke-170...
🧩 Memproses frame ke-180...
🧩 Memproses frame ke-190...
🧩 Memproses frame ke-200...
🧩 Memproses frame ke-210...
🧩 Memproses frame ke-220...
🧩 Memproses frame ke-230...
🧩 Memproses frame ke-240...
🧩 Memproses frame ke-250...
🧩 Memproses frame ke-260...
🧩 Memproses frame ke-270...
🧩 Memproses frame ke-280...
🧩 Memproses frame ke-290...
🧩 Memproses frame ke-300...
🧩 Memproses frame ke-310...
🧩 Memproses frame ke-320...
🧩 Memproses frame ke-330...
🧩 Memproses frame ke-340...
🧩 Memproses frame ke-350...
🧩 Memproses frame ke-360...
🧩